# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [73]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [74]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [75]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2218278174010,
 'week52high': 148.74,
 'week52low': 54.56,
 'week52highSplitAdjustOnly': 151.76,
 'week52lowSplitAdjustOnly': 54.99,
 'week52change': 0.7028461932559845,
 'sharesOutstanding': 16928356645,
 'float': 0,
 'avg10Volume': 83564462,
 'avg30Volume': 100521731,
 'day200MovingAvg': 122.69,
 'day50MovingAvg': 138.46,
 'employees': 152742,
 'ttmEPS': 3.7,
 'ttmDividendRate': 0.8496946978868768,
 'dividendYield': 0.006533755998193762,
 'nextDividendDate': '0',
 'exDividendDate': '2021-01-25',
 'nextEarningsDate': '2021-01-24',
 'peRatio': 34.10526523285007,
 'beta': 1.1915894958126534,
 'maxChangePercent': 49.47208131087543,
 'year5ChangePercent': 4.853782218050585,
 'year2ChangePercent': 2.15166066354078,
 'year1ChangePercent': 0.63582678251832,
 'ytdChangePercent': -0.02019925978383752,
 'month6ChangePercent': 0.12700799213888292,
 'month3ChangePercent': 0.0998124385302026,
 'month1ChangePercent': 0.01778050215140169,
 'day30ChangePerce

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [76]:
data['year1ChangePercent']

0.63582678251832

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [77]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

In [78]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['price'],
            data[symbol]['stats']['year1ChangePercent'],
            'N/A'
        ],
        index = my_columns),
            ignore_index = True
        )
final_dataframe

Exception ignored in: <function ZipFile.__del__ at 0x000001E45278AC10>
Traceback (most recent call last):
  File "C:\Users\Alex\AppData\Local\Programs\Python\Python39\lib\zipfile.py", line 1807, in __del__
    self.close()
  File "C:\Users\Alex\AppData\Local\Programs\Python\Python39\lib\zipfile.py", line 1824, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,130.93,0.509053,N/A
1,AAL,19.52,-0.354724,N/A
2,AAP,169.38,0.20219,N/A
3,AAPL,135.56,0.625444,N/A
4,ABBV,108.27,0.181628,N/A
...,...,...,...,...
500,YUM,105.72,0.019007,N/A
501,ZBH,159.31,-0.013773,N/A
502,ZBRA,512.98,1.089497,N/A
503,ZION,53.50,0.170946,N/A


Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [79]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,78,CARR,37.90,2.187317,N/A
1,179,FCX,38.84,2.102613,N/A
2,387,PYPL,301.16,1.336042,N/A
3,24,ALGN,625.92,1.167225,N/A
4,502,ZBRA,512.98,1.089497,N/A
5,275,LB,52.21,1.088484,N/A
6,128,DE,334.00,1.085329,N/A
7,410,SIVB,566.70,1.05572,N/A
8,6,ABMD,348.11,1.010409,N/A
9,266,KLAC,342.00,0.973577,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [91]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio:')
    
    try:
        float(portfolio_size)
    except ValueError:
        print('That is not a number! \nPlease try again:')
        portfolio_size = input('Enter the size of your portfolio:')
        
portfolio_input()
print(portfolio_size)

Enter the size of your portfolio:1000000
1000000


In [92]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[i, 'Price'])
    
final_dataframe

C:\Users\Alex\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,78,CARR,37.90,2.187317,527
1,179,FCX,38.84,2.102613,514
2,387,PYPL,301.16,1.336042,66
3,24,ALGN,625.92,1.167225,31
4,502,ZBRA,512.98,1.089497,38
5,275,LB,52.21,1.088484,383
6,128,DE,334.00,1.085329,59
7,410,SIVB,566.70,1.05572,35
8,6,ABMD,348.11,1.010409,57
9,266,KLAC,342.00,0.973577,58


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [93]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
            hqm_dataframe = hqm_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['price'],
                0,
                data[symbol]['stats']['year1ChangePercent'],
                0,
                data[symbol]['stats']['month6ChangePercent'],
                0,
                data[symbol]['stats']['month3ChangePercent'],
                0,
                data[symbol]['stats']['month1ChangePercent'],
                0,
                'N/A'
            ],
                index = hqm_columns),
                ignore_index = True
            )
            
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,132.46,0,0.505551,0,0.308746,0,0.174747,0,-0.0038,0,N/A
1,AAL,18.84,0,-0.344155,0,0.47703,0,0.468914,0,0.175367,0,N/A
2,AAP,168.04,0,0.200006,0,0.035768,0,0.144078,0,0.000303,0,N/A
3,AAPL,131.90,0,0.640718,0,0.129363,0,0.096307,0,0.017504,0,N/A
4,ABBV,109.84,0,0.182705,0,0.126343,0,0.066735,0,-0.06583,0,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.16,0,0.019218,0,0.13981,0,0.008046,0,-0.016623,0,N/A
501,ZBH,159.78,0,-0.013213,0,0.151106,0,0.074406,0,-0.021136,0,N/A
502,ZBRA,509.06,0,1.049537,0,0.75453,0,0.376997,0,0.210499,0,N/A
503,ZION,52.90,0,0.173561,0,0.608283,0,0.365571,0,0.05542,0,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [94]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        if hqm_dataframe.loc[row, f'{time_period} Price Return'] == None:
            hqm_dataframe.loc[row, f'{time_period} Price Return'] = 0

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100
        
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,132.46,0,0.505551,0.885149,0.308746,0.651485,0.174747,0.671287,-0.0038,0.344554,N/A
1,AAL,18.84,0,-0.344155,0.023762,0.47703,0.807921,0.468914,0.956436,0.175367,0.950495,N/A
2,AAP,168.04,0,0.200006,0.637624,0.035768,0.221782,0.144078,0.629703,0.000303,0.372277,N/A
3,AAPL,131.90,0,0.640718,0.940594,0.129363,0.415842,0.096307,0.512871,0.017504,0.489109,N/A
4,ABBV,109.84,0,0.182705,0.621782,0.126343,0.39802,0.066735,0.453465,-0.06583,0.079208,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.16,0,0.019218,0.386139,0.13981,0.441584,0.008046,0.287129,-0.016623,0.253465,N/A
501,ZBH,159.78,0,-0.013213,0.328713,0.151106,0.449505,0.074406,0.479208,-0.021136,0.223762,N/A
502,ZBRA,509.06,0,1.049537,0.988119,0.75453,0.928713,0.376997,0.918812,0.210499,0.974257,N/A
503,ZION,52.90,0,0.173561,0.60396,0.608283,0.879208,0.365571,0.90099,0.05542,0.673267,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [95]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,132.46,0,0.505551,0.885149,0.308746,0.651485,0.174747,0.671287,-0.0038,0.344554,0.638119
1,AAL,18.84,0,-0.344155,0.023762,0.47703,0.807921,0.468914,0.956436,0.175367,0.950495,0.684653
2,AAP,168.04,0,0.200006,0.637624,0.035768,0.221782,0.144078,0.629703,0.000303,0.372277,0.465347
3,AAPL,131.90,0,0.640718,0.940594,0.129363,0.415842,0.096307,0.512871,0.017504,0.489109,0.589604
4,ABBV,109.84,0,0.182705,0.621782,0.126343,0.39802,0.066735,0.453465,-0.06583,0.079208,0.388119
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.16,0,0.019218,0.386139,0.13981,0.441584,0.008046,0.287129,-0.016623,0.253465,0.342079
501,ZBH,159.78,0,-0.013213,0.328713,0.151106,0.449505,0.074406,0.479208,-0.021136,0.223762,0.370297
502,ZBRA,509.06,0,1.049537,0.988119,0.75453,0.928713,0.376997,0.918812,0.210499,0.974257,0.952475
503,ZION,52.90,0,0.173561,0.60396,0.608283,0.879208,0.365571,0.90099,0.05542,0.673267,0.764356


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [96]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace = True, drop = True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,VIAC,63.74,0,0.861268,0.970297,1.350881,0.994059,0.900473,0.994059,0.449603,0.99802,0.989109
1,FCX,39.10,0,2.19903,1.0,1.53779,0.99604,0.798977,0.990099,0.18528,0.958416,0.986139
2,DISCA,53.16,0,0.695845,0.952475,1.292403,0.992079,1.079922,1.0,0.424118,0.99604,0.985149
3,TWTR,74.35,0,0.867186,0.972277,0.868541,0.962376,0.676809,0.988119,0.58339,1.0,0.980693
4,SIVB,550.90,0,1.072487,0.990099,1.248524,0.988119,0.548027,0.970297,0.14833,0.914851,0.965842
5,DISCK,44.90,0,0.497255,0.883168,1.127622,0.982178,0.9243,0.99802,0.346647,0.994059,0.964356
6,ZBRA,509.06,0,1.049537,0.988119,0.75453,0.928713,0.376997,0.918812,0.210499,0.974257,0.952475
7,TPR,41.19,0,0.443202,0.857426,1.694858,0.99802,0.401694,0.932673,0.185351,0.960396,0.937129
8,AMAT,119.69,0,0.798582,0.966337,0.827016,0.952475,0.570218,0.974257,0.096371,0.825743,0.929703
9,PYPL,298.32,0,1.379109,0.99604,0.491719,0.813861,0.504077,0.966337,0.162377,0.940594,0.929208


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [98]:
portfolio_input()

Enter the size of your portfolio:
That is not a number! 
Please try again:
Enter the size of your portfolio:1000000


In [99]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])
    
hqm_dataframe

C:\Users\Alex\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,VIAC,63.74,313,0.861268,0.970297,1.350881,0.994059,0.900473,0.994059,0.449603,0.99802,0.989109
1,FCX,39.10,511,2.19903,1.0,1.53779,0.99604,0.798977,0.990099,0.18528,0.958416,0.986139
2,DISCA,53.16,376,0.695845,0.952475,1.292403,0.992079,1.079922,1.0,0.424118,0.99604,0.985149
3,TWTR,74.35,268,0.867186,0.972277,0.868541,0.962376,0.676809,0.988119,0.58339,1.0,0.980693
4,SIVB,550.90,36,1.072487,0.990099,1.248524,0.988119,0.548027,0.970297,0.14833,0.914851,0.965842
5,DISCK,44.90,445,0.497255,0.883168,1.127622,0.982178,0.9243,0.99802,0.346647,0.994059,0.964356
6,ZBRA,509.06,39,1.049537,0.988119,0.75453,0.928713,0.376997,0.918812,0.210499,0.974257,0.952475
7,TPR,41.19,485,0.443202,0.857426,1.694858,0.99802,0.401694,0.932673,0.185351,0.960396,0.937129
8,AMAT,119.69,167,0.798582,0.966337,0.827016,0.952475,0.570218,0.974257,0.096371,0.825743,0.929703
9,PYPL,298.32,67,1.379109,0.99604,0.491719,0.813861,0.504077,0.966337,0.162377,0.940594,0.929208


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [100]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name = 'Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [101]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [102]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['One-Year Price Return', percent_template],
    'E': ['One-Year Return Percentile', percent_template],
    'F': ['Six-Month Price Return', percent_template],
    'G': ['Six-Month Return Percentile', percent_template],
    'H': ['Three-Month Price Return', percent_template],
    'I': ['Three-Month Return Percentile', percent_template],
    'J': ['One-Month Price Return', percent_template],
    'K': ['One-Month Return Percentile', percent_template],
    'L': ['HQM Score', percent_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
    
writer.save()

## Saving Our Excel Output

As before, saving our Excel output is very easy: